<center><h1>Geng_Haoxiang_Final_Project</h1></center>
<br>

|      Name      |  GitHub Username  |   USC ID   |
|----------------|-------------------|------------|
|  Haoxiang Geng |    Haoxiang310    | 8045015278 |

In [1]:
!pip install tensorflow
!pip install opencv-python
!pip install tensorflow-metal
import os
import os.path as op
import json
from pathlib import Path
import shutil
import logging
import numpy as np
from tqdm import tqdm
from skimage import io
import random
import tensorflow as tf
from PIL import Image 
import cv2
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from tensorflow.keras.applications import EfficientNetB0, ResNet50, VGG16
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

## (b) Data Exploration and Pre-processing

In [2]:
# Logging configuration
logging.basicConfig(level=logging.INFO,
                    datefmt='%H:%M:%S',
                    format='%(asctime)s | %(levelname)-5s | %(module)-15s | %(message)s')

IMAGE_SIZE = (299, 299)  # All images contained in this dataset are 299x299 (originally, to match Inception v3 input size)
SEED = 17

# Head directory containing all image subframes. Update with the relative path of your data directory
data_head_dir = Path('./data')

# Find all subframe directories
subdirs = [Path(subdir.stem) for subdir in data_head_dir.iterdir() if subdir.is_dir()]
src_image_ids = ['_'.join(a_path.name.split('_')[:3]) for a_path in subdirs]

In [3]:
# Load train/val/test subframe IDs
def load_text_ids(file_path):
    """Simple helper to load all lines from a text file"""
    with open(file_path, 'r') as f:
        lines = [line.strip() for line in f.readlines()]
    return lines

# Load the subframe names for the three data subsets
train_ids = load_text_ids('./train_source_images.txt')
validate_ids = load_text_ids('./val_source_images.txt')
test_ids = load_text_ids('./test_source_images.txt')

# Generate a list containing the dataset split for the matching subdirectory names
subdir_splits = []
for src_id in src_image_ids:
    if src_id in train_ids:
        subdir_splits.append('train')
    elif src_id in validate_ids:
        subdir_splits.append('validate')
    elif(src_id in test_ids):
        subdir_splits.append('test')
    else:
        logging.warning(f'{src_id}: Did not find designated split in train/validate/test list.')
        subdir_splits.append(None)

In [4]:
def load_and_preprocess(img_loc, label):
    def _inner_function(img_loc, label):
        
        #data augmentation
        img_loc_str = img_loc.numpy().decode('utf-8')
        label_str = label.numpy().decode('utf-8')
        img = cv2.imread(img_loc_str)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (299, 299))
        if random.random() > 0.5:
            img = cv2.flip(img, 1) 
        if random.random() > 0.5:
            img = cv2.flip(img, 0)
        angle = random.randint(-15, 15)
        M = cv2.getRotationMatrix2D((img.shape[1]//2, img.shape[0]//2), angle, 1)
        img = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]))
        alpha = 1.0 + random.uniform(-0.3, 0.3) 
        beta = random.uniform(-30, 30)
        img = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)
        img = tf.convert_to_tensor(img, dtype=tf.float32)
        img = img / 255.0
        img.set_shape([299, 299, 3])
        label = tf.cast(1 if label_str == 'frost' else 0, tf.int32)
        label.set_shape([])
        return img, label

    X, y = tf.py_function(_inner_function, [img_loc, label], [tf.float32, tf.int32])
    X.set_shape([299, 299, 3])
    y.set_shape([])
    return X, y



def load_subdir_data(dir_path, image_size, seed=None):
    
    """Helper to create a TF dataset from each image subdirectory"""
    
    # Grab only the classes that (1) we want to keep and (2) exist in this directory
    tile_dir = dir_path / Path('tiles')
    label_dir = dir_path /Path('labels')
    
    loc_list = []
    
    for folder in os.listdir(tile_dir):
        if os.path.isdir(os.path.join(tile_dir, folder)):
            for file in os.listdir(os.path.join(tile_dir, folder)):
                if file.endswith(".png"):
                    loc_list.append((os.path.join(os.path.join(tile_dir, folder), file), folder))

    return loc_list

# Loop over all subframes, loading each into a list
tf_data_train, tf_data_test, tf_data_val = [], [], []
tf_dataset_train, tf_dataset_test, tf_dataset_val = [], [], []

# Update the batch and buffer size as per your model requirements
buffer_size = 64
batch_size = 32

for subdir, split in zip(subdirs, subdir_splits):
    full_path = data_head_dir / subdir
    if split=='validate':
        tf_data_val.extend(load_subdir_data(full_path, IMAGE_SIZE, SEED))
    elif split=='train':
        tf_data_train.extend(load_subdir_data(full_path, IMAGE_SIZE, SEED))
    elif split=='test':
        tf_data_test.extend(load_subdir_data(full_path, IMAGE_SIZE, SEED))
        
random.shuffle(tf_data_train)
img_list, label_list = zip(*tf_data_train)
img_list_t = tf.convert_to_tensor(img_list)
lb_list_t = tf.convert_to_tensor(label_list)

tf_dataset_train = tf.data.Dataset.from_tensor_slices((img_list_t, lb_list_t))
tf_dataset_train = tf_dataset_train.map(load_and_preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)
tf_dataset_train = tf_dataset_train.shuffle(buffer_size=buffer_size).batch(batch_size) 

random.shuffle(tf_data_val)
img_list, label_list = zip(*tf_data_val)
img_list_t = tf.convert_to_tensor(img_list)
lb_list_t = tf.convert_to_tensor(label_list)

tf_dataset_val = tf.data.Dataset.from_tensor_slices((img_list_t, lb_list_t))
tf_dataset_val = tf_dataset_val.map(load_and_preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)
tf_dataset_val = tf_dataset_val.shuffle(buffer_size=buffer_size).batch(batch_size) 

random.shuffle(tf_data_test)
img_list, label_list = zip(*tf_data_test)
img_list_t = tf.convert_to_tensor(img_list)
lb_list_t = tf.convert_to_tensor(label_list)

tf_dataset_test = tf.data.Dataset.from_tensor_slices((img_list_t, lb_list_t))
tf_dataset_test = tf_dataset_test.map(load_and_preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)
tf_dataset_test = tf_dataset_test.shuffle(buffer_size=buffer_size).batch(batch_size) 

2023-12-08 18:20:16.503151: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2023-12-08 18:20:16.503210: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2023-12-08 18:20:16.503250: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2023-12-08 18:20:16.503332: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-12-08 18:20:16.503362: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
num_batches_train = tf.data.experimental.cardinality(tf_dataset_train).numpy()
num_batches_val = tf.data.experimental.cardinality(tf_dataset_val).numpy()
num_batches_test = tf.data.experimental.cardinality(tf_dataset_test).numpy()

total_tiles_train = num_batches_train * batch_size
total_tiles_val = num_batches_val * batch_size
total_tiles_test = num_batches_test * batch_size

print(f"Total number of tiles in the training dataset: {total_tiles_train}")
print(f"Total number of tiles in the validation dataset: {total_tiles_val}")
print(f"Total number of tiles in the test dataset: {total_tiles_test}")


Total number of tiles in the training dataset: 29696
Total number of tiles in the validation dataset: 11296
Total number of tiles in the test dataset: 12832


## (C) Training CNN + MLP

In [6]:
def build_model():
    model = models.Sequential()
    model.add(layers.InputLayer(input_shape=(299, 299, 3)))
    model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(2, activation='softmax'))

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

model = build_model()


In [7]:
def evaluate_and_plot(model, history, test_dataset, title, class_names=['Background', 'Frost']):
    y_pred = model.predict(test_dataset)
    y_pred = np.argmax(y_pred, axis=1)
    y_true = []
    for _, label in test_dataset.unbatch():
        y_true.append(label.numpy())
    y_true = np.array(y_true)
    report = classification_report(y_true, y_pred, target_names=class_names)
    print(report)
    
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.legend()
    plt.title(f'{title} - Accuracy over epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.legend()
    plt.title(f'{title} - Loss over epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')

    plt.show()

In [ ]:
history = model.fit(tf_dataset_train, 
                    epochs=20, 
                    validation_data=tf_dataset_val,
                    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)])
evaluate_and_plot(model, history, tf_dataset_test, 'CNN')

Epoch 1/20


2023-12-08 18:20:16.993591: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


928/928 [==============================] - 152s 164ms/step - loss: 0.7767 - accuracy: 0.5737 - val_loss: 0.7394 - val_accuracy: 0.3218
Epoch 2/20
928/928 [==============================] - 152s 164ms/step - loss: 0.6828 - accuracy: 0.5795 - val_loss: 0.6950 - val_accuracy: 0.3297
Epoch 3/20
928/928 [==============================] - 152s 164ms/step - loss: 0.6618 - accuracy: 0.5984 - val_loss: 0.6768 - val_accuracy: 0.5933
Epoch 4/20
928/928 [==============================] - 152s 163ms/step - loss: 0.6572 - accuracy: 0.6045 - val_loss: 0.6999 - val_accuracy: 0.3744
Epoch 5/20
928/928 [==============================] - 1085s 1s/step - loss: 0.6553 - accuracy: 0.6090 - val_loss: 0.6818 - val_accuracy: 0.5269
Epoch 6/20
642/928 [===================>..........] - ETA: 40s - loss: 0.6570 - accuracy: 0.6083

## (d) Transfer learning

In [ ]:
def create_transfer_model(model_name):
    if model_name == 'EfficientNetB0':
        base_model = EfficientNetB0(include_top=False, input_shape=(299, 299, 3), weights='imagenet')
    elif model_name == 'ResNet50':
        base_model = ResNet50(include_top=False, input_shape=(299, 299, 3), weights='imagenet')
    elif model_name == 'VGG16':
        base_model = VGG16(include_top=False, input_shape=(299, 299, 3), weights='imagenet')
    else:
        raise ValueError("Unsupported model name")

    # freeze
    base_model.trainable = False 

    inputs = tf.keras.Input(shape=(299, 299, 3))
    x = base_model(inputs, training=False)
    x = GlobalAveragePooling2D()(x)
    x = BatchNormalization()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.3)(x)
    outputs = Dense(2, activation='softmax')(x)

    model = Model(inputs, outputs)
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

model_efficientnet = create_transfer_model('EfficientNetB0')

history_efficientnet = model_efficientnet.fit(tf_dataset_train, epochs=20, validation_data=tf_dataset_val, callbacks=[early_stopping_cb])

evaluate_and_plot(model_efficientnet, history_efficientnet, tf_dataset_test, 'EfficientNetB0')

In [ ]:
model_resnet = create_transfer_model('ResNet50')

history_resnet = model_resnet.fit(tf_dataset_train, epochs=20, validation_data=tf_dataset_val, callbacks=[early_stopping_cb])

evaluate_and_plot(model_resnet, history_resnet, tf_dataset_test, 'ResNet50')

In [ ]:
model_vgg16 = create_transfer_model('VGG16')

history_vgg16 = model_vgg16.fit(tf_dataset_train, epochs=20, validation_data=tf_dataset_val, callbacks=[early_stopping_cb])

evaluate_and_plot(model_vgg16, history_vgg16, tf_dataset_test, 'VGG16')

### (vi).Compare results: 
From the perspective of training accuracy and test accuracy, transfer learning outperforms CNN + MLP. Within transfer learning approaches, VGG16 is the best, followed by ResNet50, with EfficientNet being the least effective. This is because transfer learning is able to leverage pre-trained networks. These networks have already learned robust and complex feature representations from large and diverse datasets, typically on tasks like ImageNet classification. When applying transfer learning, this rich feature understanding is adapted to new, possibly smaller, datasets, providing a head start in learning. In contrast, a basic three-layer CNN+MLP model starts learning from scratch, which can be less efficient and effective, especially with limited data.